In [1]:
import pandas as pd
import numpy as np
from usfull_tools import load_DS
from catboost import Pool, CatBoostRegressor, CatBoostClassifier
from sklearn.model_selection import train_test_split
pd.options.display.max_columns = None
%matplotlib inline

from set_vars import KAGGLE_PREFIX, debug_mode, KAGGLE_DIR, target_column, target_type, loss_function, custom_metric

train, test = load_DS(debug_mode, KAGGLE_DIR, KAGGLE_PREFIX, '_prepare.csv')
del test

X_train, X_test, y_train, y_test = train_test_split(train[train.columns.drop(target_column)], train[target_column], 
                                                    test_size=0.3, random_state=42)
X_train.shape

(1020, 292)

In [8]:
# iterations = np.round(10+len(X_train.columns)/2)
iterations = 300


    
#https://tech.yandex.com/catboost/doc/dg/concepts/python-reference_parameters-list-docpage/#python-reference_parameters-list

# Для CatBoost требуется явно указывать категориальные переменные
i=0
cat_features = []
for column in X_train.columns:
    if X_train[column].dtype == 'object': cat_features.append(i)
    i +=1

    
if target_type=='binary':
    model = CatBoostClassifier(random_seed = 42, iterations=iterations, depth=2, learning_rate=0.1, 
                               loss_function=loss_function, od_pval = 1e-3, cat_features = cat_features
#                               , task_type='GPU', devices='0'
                              )
elif target_type=='interval':
    model = CatBoostRegressor(random_seed = 42, iterations=iterations, depth=6, learning_rate=0.1, 
                              loss_function=loss_function, od_pval = 1e-3, cat_features = cat_features
                              )
    
model.fit(X_train, y_train)

from sklearn.metrics import mean_absolute_error, accuracy_score

if custom_metric=='Accuracy':
    print("Accuracy: %.3f"
          % accuracy_score(model.predict(X_test), y_test))

if custom_metric=='RMSE':
    print("RMSE: %.3f"
          % mean_absolute_error(model.predict(X_test), y_test))


feature_importance = pd.DataFrame(list(zip(X_test.dtypes.index, 
                                           model.get_feature_importance(Pool(X_test, label=y_test, cat_features=cat_features)))),
                                    columns=['Feature','Score'])

feature_importance = feature_importance.sort_values(by='Score', ascending=False, inplace=False, kind='quicksort', na_position='last')

#TO DO: сделать отбор лучшего из нескольких корелирующих параметров

0:	learn: 180449.3598235	total: 171ms	remaining: 51.2s
1:	learn: 164352.6367270	total: 290ms	remaining: 43.3s
2:	learn: 150520.5392577	total: 405ms	remaining: 40.1s
3:	learn: 137979.0330630	total: 556ms	remaining: 41.1s
4:	learn: 127072.2293413	total: 679ms	remaining: 40s
5:	learn: 116974.3893491	total: 794ms	remaining: 38.9s
6:	learn: 107640.9181104	total: 910ms	remaining: 38.1s
7:	learn: 99586.1260040	total: 1.03s	remaining: 37.6s
8:	learn: 91843.1990593	total: 1.15s	remaining: 37.2s
9:	learn: 85407.1487006	total: 1.29s	remaining: 37.3s
10:	learn: 79529.3607386	total: 1.42s	remaining: 37.2s
11:	learn: 74381.4489987	total: 1.54s	remaining: 37.1s
12:	learn: 69646.0952277	total: 1.67s	remaining: 36.9s
13:	learn: 65071.8757779	total: 1.79s	remaining: 36.7s
14:	learn: 61299.3672258	total: 1.91s	remaining: 36.3s
15:	learn: 57848.1522713	total: 2.02s	remaining: 35.9s
16:	learn: 54835.4665865	total: 2.14s	remaining: 35.7s
17:	learn: 51835.3407647	total: 2.26s	remaining: 35.4s
18:	learn: 4928

151:	learn: 19299.3406820	total: 17.2s	remaining: 16.7s
152:	learn: 19280.9078128	total: 17.3s	remaining: 16.6s
153:	learn: 19235.7680076	total: 17.4s	remaining: 16.5s
154:	learn: 19229.0865471	total: 17.5s	remaining: 16.4s
155:	learn: 19221.9058185	total: 17.6s	remaining: 16.3s
156:	learn: 19212.4737078	total: 17.7s	remaining: 16.2s
157:	learn: 19195.3869263	total: 17.8s	remaining: 16s
158:	learn: 19183.8121670	total: 18s	remaining: 15.9s
159:	learn: 19168.8366872	total: 18.1s	remaining: 15.8s
160:	learn: 19159.8605415	total: 18.2s	remaining: 15.7s
161:	learn: 19141.0459333	total: 18.3s	remaining: 15.6s
162:	learn: 19118.9485373	total: 18.4s	remaining: 15.5s
163:	learn: 19105.5859030	total: 18.5s	remaining: 15.3s
164:	learn: 19095.8550191	total: 18.6s	remaining: 15.2s
165:	learn: 19069.2198535	total: 18.7s	remaining: 15.1s
166:	learn: 19062.8826903	total: 18.8s	remaining: 15s
167:	learn: 19049.8258062	total: 18.9s	remaining: 14.9s
168:	learn: 19039.5018062	total: 19.1s	remaining: 14.8

299:	learn: 16835.7759153	total: 34.2s	remaining: 0us


In [15]:
#Keep features with > 0.1% normalize importance
fi = feature_importance[feature_importance.Score > 0.01]
print(len(X_train.columns), '->', len(fi.index), 'non zero important features:', np.round(fi.Score.sum(),1), '%')
fi.sort_values('Score', ascending = False)

292 -> 129 non zero important features: 99.9 %


,Feature,Score
4,OverallQual,15.502309
16,GrLivArea,13.393390
190,ExterQual_dummie_TA,10.055163
12,TotalBsmtSF,5.756748
27,GarageCars,4.319288
6,YearBuilt,4.225956
244,KitchenQual_dummie_Ex,3.772998
9,BsmtFinSF1,3.325598
24,Fireplaces,3.174879
13,1stFlrSF,2.252342


In [4]:
# from sklearn.feature_selection import RFE

# selector = RFE(model)
# selector = selector.fit(X_train, y_train)

0:	learn: 180449.3598235	total: 162ms	remaining: 4.71s
1:	learn: 164352.6367270	total: 275ms	remaining: 3.84s
2:	learn: 150520.5392577	total: 384ms	remaining: 3.46s
3:	learn: 137979.0330630	total: 501ms	remaining: 3.25s
4:	learn: 127072.2293413	total: 617ms	remaining: 3.09s
5:	learn: 116974.3893491	total: 737ms	remaining: 2.95s
6:	learn: 107640.9181104	total: 852ms	remaining: 2.8s
7:	learn: 99586.1260040	total: 979ms	remaining: 2.69s
8:	learn: 91843.1990593	total: 1.1s	remaining: 2.56s
9:	learn: 85407.1487006	total: 1.21s	remaining: 2.42s
10:	learn: 79529.3607386	total: 1.32s	remaining: 2.28s
11:	learn: 74381.4489987	total: 1.44s	remaining: 2.15s
12:	learn: 69646.0952277	total: 1.56s	remaining: 2.04s
13:	learn: 65071.8757779	total: 1.69s	remaining: 1.93s
14:	learn: 61299.3672258	total: 1.83s	remaining: 1.83s
15:	learn: 57848.1522713	total: 1.96s	remaining: 1.71s
16:	learn: 54835.4665865	total: 2.07s	remaining: 1.58s
17:	learn: 51835.3407647	total: 2.19s	remaining: 1.46s
18:	learn: 4928

0:	learn: 179963.4025625	total: 135ms	remaining: 3.9s
1:	learn: 164634.9311846	total: 284ms	remaining: 3.98s
2:	learn: 150441.6569167	total: 439ms	remaining: 3.95s
3:	learn: 138156.4925744	total: 589ms	remaining: 3.83s
4:	learn: 127098.2364071	total: 753ms	remaining: 3.77s
5:	learn: 116706.1335755	total: 904ms	remaining: 3.61s
6:	learn: 107625.1438765	total: 1.06s	remaining: 3.47s
7:	learn: 99309.6805141	total: 1.21s	remaining: 3.32s
8:	learn: 92094.0177206	total: 1.36s	remaining: 3.18s
9:	learn: 85186.6327134	total: 1.51s	remaining: 3.03s
10:	learn: 79130.8326559	total: 1.67s	remaining: 2.89s
11:	learn: 74093.5373304	total: 1.82s	remaining: 2.73s
12:	learn: 69030.4797768	total: 1.98s	remaining: 2.59s
13:	learn: 64640.3456494	total: 2.12s	remaining: 2.43s
14:	learn: 60523.3274471	total: 2.29s	remaining: 2.29s
15:	learn: 57143.0397364	total: 2.43s	remaining: 2.13s
16:	learn: 54098.8682017	total: 2.61s	remaining: 2s
17:	learn: 51525.4525469	total: 2.76s	remaining: 1.84s
18:	learn: 49003.

2:	learn: 150641.7212080	total: 456ms	remaining: 4.1s
3:	learn: 137972.2932762	total: 601ms	remaining: 3.9s
4:	learn: 126789.9202471	total: 760ms	remaining: 3.8s
5:	learn: 116598.7438731	total: 914ms	remaining: 3.65s
6:	learn: 107474.7118894	total: 1.07s	remaining: 3.5s
7:	learn: 98928.7022795	total: 1.22s	remaining: 3.35s
8:	learn: 91473.5668881	total: 1.37s	remaining: 3.21s
9:	learn: 84627.7627583	total: 1.52s	remaining: 3.04s
10:	learn: 78628.9255893	total: 1.68s	remaining: 2.9s
11:	learn: 73470.4181708	total: 1.83s	remaining: 2.75s
12:	learn: 68465.7665018	total: 2s	remaining: 2.61s
13:	learn: 64246.5821395	total: 2.15s	remaining: 2.46s
14:	learn: 60141.1577518	total: 2.31s	remaining: 2.31s
15:	learn: 56679.9156992	total: 2.46s	remaining: 2.15s
16:	learn: 53678.9639922	total: 2.62s	remaining: 2s
17:	learn: 50920.2507467	total: 2.77s	remaining: 1.84s
18:	learn: 48601.5951538	total: 2.91s	remaining: 1.69s
19:	learn: 46455.2471979	total: 3.06s	remaining: 1.53s
20:	learn: 44710.0567066

4:	learn: 127553.9703495	total: 815ms	remaining: 4.07s
5:	learn: 117037.0084012	total: 980ms	remaining: 3.92s
6:	learn: 107823.9070425	total: 1.13s	remaining: 3.7s
7:	learn: 100109.8181560	total: 1.29s	remaining: 3.55s
8:	learn: 92431.5820491	total: 1.45s	remaining: 3.39s
9:	learn: 85803.3213621	total: 1.61s	remaining: 3.21s
10:	learn: 80140.7918294	total: 1.78s	remaining: 3.08s
11:	learn: 74714.8063866	total: 1.94s	remaining: 2.91s
12:	learn: 69612.7282489	total: 2.1s	remaining: 2.74s
13:	learn: 64837.1034815	total: 2.24s	remaining: 2.56s
14:	learn: 61207.7884518	total: 2.39s	remaining: 2.39s
15:	learn: 57880.7250172	total: 2.53s	remaining: 2.22s
16:	learn: 54875.7632181	total: 2.69s	remaining: 2.06s
17:	learn: 51797.9756709	total: 2.86s	remaining: 1.91s
18:	learn: 49227.3195902	total: 3.02s	remaining: 1.75s
19:	learn: 46899.2022421	total: 3.16s	remaining: 1.58s
20:	learn: 44988.1905538	total: 3.31s	remaining: 1.42s
21:	learn: 43296.4229866	total: 3.46s	remaining: 1.26s
22:	learn: 416

6:	learn: 108414.2397508	total: 1.05s	remaining: 3.46s
7:	learn: 99841.6033439	total: 1.21s	remaining: 3.32s
8:	learn: 92546.9854228	total: 1.35s	remaining: 3.15s
9:	learn: 85827.8133845	total: 1.5s	remaining: 2.99s
10:	learn: 79744.4497859	total: 1.64s	remaining: 2.83s
11:	learn: 74234.2218569	total: 1.79s	remaining: 2.68s
12:	learn: 69139.3704107	total: 1.95s	remaining: 2.55s
13:	learn: 65075.6438146	total: 2.02s	remaining: 2.31s
14:	learn: 61251.1981042	total: 2.16s	remaining: 2.16s
15:	learn: 57748.9274238	total: 2.31s	remaining: 2.02s
16:	learn: 54565.5559515	total: 2.47s	remaining: 1.89s
17:	learn: 51567.0727742	total: 2.62s	remaining: 1.75s
18:	learn: 49157.3417658	total: 2.77s	remaining: 1.6s
19:	learn: 46553.2681825	total: 2.92s	remaining: 1.46s
20:	learn: 44696.7746979	total: 3.07s	remaining: 1.31s
21:	learn: 42673.7170490	total: 3.21s	remaining: 1.17s
22:	learn: 41047.6326953	total: 3.36s	remaining: 1.02s
23:	learn: 39312.7632817	total: 3.51s	remaining: 877ms
24:	learn: 3803

...

20:	learn: 44668.9113631	total: 2.48s	remaining: 1.06s
21:	learn: 43044.0137968	total: 2.6s	remaining: 944ms
22:	learn: 41557.8026667	total: 2.71s	remaining: 824ms
23:	learn: 40018.9176339	total: 2.81s	remaining: 704ms
24:	learn: 38630.0971813	total: 2.94s	remaining: 587ms
25:	learn: 37704.6519497	total: 3.08s	remaining: 474ms
26:	learn: 36549.4569532	total: 3.2s	remaining: 356ms
27:	learn: 35899.7432955	total: 3.32s	remaining: 237ms
28:	learn: 35149.1917155	total: 3.43s	remaining: 118ms
29:	learn: 34564.4266024	total: 3.55s	remaining: 0us
0:	learn: 180237.6728133	total: 105ms	remaining: 3.03s
1:	learn: 164736.6714348	total: 222ms	remaining: 3.11s
2:	learn: 150925.5477770	total: 364ms	remaining: 3.27s
3:	learn: 139123.2086505	total: 480ms	remaining: 3.12s
4:	learn: 127622.0347584	total: 589ms	remaining: 2.94s
5:	learn: 117112.7720764	total: 715ms	remaining: 2.86s
6:	learn: 108044.9713274	total: 836ms	remaining: 2.75s
7:	learn: 99785.1746467	total: 960ms	remaining: 2.64s
8:	learn: 91960

21:	learn: 41770.0994198	total: 2.57s	remaining: 933ms
22:	learn: 40287.1577576	total: 2.69s	remaining: 819ms
23:	learn: 38896.3443994	total: 2.81s	remaining: 702ms
24:	learn: 37674.2769275	total: 2.92s	remaining: 583ms
25:	learn: 36645.3319129	total: 3.02s	remaining: 465ms
26:	learn: 35674.0777056	total: 3.15s	remaining: 349ms
27:	learn: 34744.6707979	total: 3.25s	remaining: 232ms
28:	learn: 33931.0903586	total: 3.36s	remaining: 116ms
29:	learn: 33302.3163504	total: 3.48s	remaining: 0us
0:	learn: 180648.9970701	total: 93ms	remaining: 2.69s
1:	learn: 164777.1140927	total: 206ms	remaining: 2.88s
2:	learn: 150422.3057793	total: 324ms	remaining: 2.91s
3:	learn: 137971.4750249	total: 431ms	remaining: 2.8s
4:	learn: 127137.6651834	total: 539ms	remaining: 2.7s
5:	learn: 116641.5807046	total: 647ms	remaining: 2.59s
6:	learn: 107707.2015672	total: 722ms	remaining: 2.37s
7:	learn: 99586.8284742	total: 830ms	remaining: 2.28s
8:	learn: 92211.1711269	total: 948ms	remaining: 2.21s
9:	learn: 85510.1

22:	learn: 40466.9201977	total: 2.62s	remaining: 797ms
23:	learn: 39156.7689752	total: 2.73s	remaining: 682ms
24:	learn: 37805.3528752	total: 2.85s	remaining: 570ms
25:	learn: 36871.7122343	total: 2.97s	remaining: 458ms
26:	learn: 35918.3729899	total: 3.06s	remaining: 340ms
27:	learn: 35030.1580031	total: 3.17s	remaining: 227ms
28:	learn: 34288.7674764	total: 3.29s	remaining: 114ms
29:	learn: 33550.1857568	total: 3.42s	remaining: 0us
0:	learn: 180545.1259167	total: 101ms	remaining: 2.92s
1:	learn: 165042.7449690	total: 222ms	remaining: 3.11s
2:	learn: 150858.5950732	total: 347ms	remaining: 3.12s
3:	learn: 138170.3176615	total: 466ms	remaining: 3.03s
4:	learn: 126774.8222607	total: 571ms	remaining: 2.86s
5:	learn: 116557.9035220	total: 682ms	remaining: 2.73s
6:	learn: 107687.0780697	total: 806ms	remaining: 2.65s
7:	learn: 98943.1897858	total: 934ms	remaining: 2.57s
8:	learn: 91303.8498155	total: 1.05s	remaining: 2.45s
9:	learn: 84421.1844372	total: 1.18s	remaining: 2.37s
10:	learn: 7817

22:	learn: 41208.0811545	total: 2.62s	remaining: 796ms
23:	learn: 40065.8698281	total: 2.72s	remaining: 681ms
24:	learn: 38792.9370020	total: 2.83s	remaining: 566ms
25:	learn: 37836.4670598	total: 2.94s	remaining: 452ms
26:	learn: 36801.5085168	total: 3.04s	remaining: 338ms
27:	learn: 35915.0690444	total: 3.15s	remaining: 225ms
28:	learn: 35051.0632248	total: 3.28s	remaining: 113ms
29:	learn: 34272.2835099	total: 3.38s	remaining: 0us
0:	learn: 179909.4452352	total: 102ms	remaining: 2.95s
1:	learn: 164067.6000620	total: 222ms	remaining: 3.11s
2:	learn: 150039.5563283	total: 297ms	remaining: 2.67s
3:	learn: 137602.9386808	total: 409ms	remaining: 2.66s
4:	learn: 126759.5734883	total: 526ms	remaining: 2.63s
5:	learn: 116671.2306418	total: 636ms	remaining: 2.54s
6:	learn: 107319.1702988	total: 759ms	remaining: 2.49s
7:	learn: 98668.4792528	total: 867ms	remaining: 2.38s
8:	learn: 91040.2621083	total: 982ms	remaining: 2.29s
9:	learn: 84491.2243778	total: 1.1s	remaining: 2.2s
10:	learn: 78488.

23:	learn: 39829.3593572	total: 2.65s	remaining: 662ms
24:	learn: 38535.1312173	total: 2.76s	remaining: 552ms
25:	learn: 37613.0534384	total: 2.88s	remaining: 442ms
26:	learn: 36592.1277154	total: 2.99s	remaining: 332ms
27:	learn: 35671.6858135	total: 3.09s	remaining: 221ms
28:	learn: 35018.2264303	total: 3.2s	remaining: 110ms
29:	learn: 34257.2465152	total: 3.31s	remaining: 0us
0:	learn: 180724.2453726	total: 97ms	remaining: 2.81s
1:	learn: 164925.2915979	total: 208ms	remaining: 2.91s
2:	learn: 151143.0246408	total: 312ms	remaining: 2.81s
3:	learn: 138597.3891333	total: 420ms	remaining: 2.73s
4:	learn: 127673.6703489	total: 530ms	remaining: 2.65s
5:	learn: 117629.6789824	total: 635ms	remaining: 2.54s
6:	learn: 108041.2211331	total: 744ms	remaining: 2.44s
7:	learn: 100036.5615597	total: 848ms	remaining: 2.33s
8:	learn: 92827.0452757	total: 967ms	remaining: 2.25s
9:	learn: 86156.1524372	total: 1.08s	remaining: 2.15s
10:	learn: 79637.9759014	total: 1.19s	remaining: 2.05s
11:	learn: 74069

In [11]:
# rfe_score = pd.DataFrame({
#     'support': selector.support_,
#     'rank': selector.ranking_,
#     'Feature': X_train.columns
# })

# rfe_score = rfe_score.sort_values('rank', ascending = True)
# fi = rfe_score[rfe_score['rank'] == 1]
# rfe_score

,support,rank,Feature
239,True,1,Electrical_dummie_FuseA
231,True,1,Heating_dummie_Wall
230,True,1,Heating_dummie_OthW
229,True,1,Heating_dummie_Grav
228,True,1,Heating_dummie_GasW
227,True,1,Heating_dummie_GasA
226,True,1,Heating_dummie_Floor
225,True,1,BsmtFinType2_dummie_Unf
224,True,1,BsmtFinType2_dummie_Rec
223,True,1,BsmtFinType2_dummie_LwQ


In [16]:
fi.to_csv(KAGGLE_DIR + KAGGLE_PREFIX + '_important_columns.csv', index=False)